# K Means Algorithm

## Import Necessary Modules

In [16]:
import numpy as np
import os
import math
import random

## Load Data to Matrix and Map it to Numeric Values

In [17]:
# Load the file and convert it to numpy matrix.

# Load necessary modules
import numpy as np
import csv



def load_text_data(file, delimiter, header, label_col_is_str , label_col):
    """
    Loading the text data and convert it to numpy matrix.
    If there is a label column with string data type, it will convert the label to Intiger and replace the string. And also return
    the mapping of the string mapping in a dictionary.
    
    Input:
    file: file name
    delimiter: "," or "\t" etc.
    header : True / False
    label_col_is_str: True/ False
    label_col : the column index of the label.
    
    Output:
    
    Data: in numpy matrix.
    label_map: in dictionary.
    
    """
    
    text_file = open(file, "rt")
    reader = csv.reader(text_file, delimiter = delimiter)
    x = list(reader)
    text_file.close()
    
    # if there is a header, remove it.
    if header:
        x.pop(0)
    Data = np.asmatrix(x)
    
    if label_col_is_str:
        try:
            Data, Label_Map = map_label(Data, label_col)
            return Data, Label_Map
        except ValueError:
            print("The header is string data type. Please specify the header = True in the function.")
            Data = None
    else:
        return Data
            




# mapping of the string labels.


# mapping the label
def map_label(Data, label_column):
    """
    If the label column is string, create a map for the label and convert it to intiger.
    
    Input:
    1. Data: in numpy matrix.
    2. label_column: the column number containing the label.
    
    Output:
    1. Data: in numpy matrix in float data type.
    """
    Label_Map = {}
    
    Label = np.unique(np.array(Data[:, label_column]))
    
    for i in range(len(Label)):
        Label_Map[Label[i]] = i
    
    mapping = lambda label, label_map : label_map[label]
    
    Data[:, label_column] = np.vectorize(mapping)(Data[:, label_column], Label_Map)
    
    Data = Data.astype(float) 
    
    return Data, Label_Map

## Split the Data to Train and Test

In [18]:
# train-test data
class split_train_test:

    
    
    def __init__(self, data, percentage, label):
        '''
        Input:
        data: a numpy matrix.
        percentage: percentage of samples for the training. percentage must be greater than 0 and less than 100.
        label: column index of the label.
        
        
        '''

        # import necessary module

        import numpy as np

        # make an one dimensional matrix n*1 to a vecto

        make_vector = lambda X : np.squeeze(np.asarray(X))


        if percentage <= 0 or percentage >= 100:
            print("The percentage of training data must be greater than 0 and less than 100.")
        else:
            import random;
        
            n = data.shape[0]  # take the number of rows in original data.
            l = int(n*percentage/100)
        
        
            # Take random integers to slice the data
            train_rows = random.sample(range(n), l)
            test_rows  = [i for i in range(n) if i not in train_rows]
            
            # Split the data to train and test.
            self.train = data[train_rows,:]
            self.test  = data[test_rows, :]
            
            # Split both train and test data to feature and label.
            self.train_feature = np.delete(self.train, label, 1)
            self.train_label = make_vector(self.train[:, label])
            
            # Split both train and test data to feature and label.
            self.test_feature = np.delete(self.test, label, 1)
            self.test_label = make_vector(self.test[:, label])
        

## K Means Algorithm

In [19]:
import numpy as np
import math


def k_means(feature_sample, test_sample, k, iter_conv):
    """
    Input:
    feature_sample: The training features without the label.
    test_sample: The test features without the label.
    k: number of clusters needed.
    iter_conv: the limit of iteration for convergence. The number of iteration will not pass this limit. When the limit is reached
    the result is produced.
    
    Output:
    centroids: The cluster centroids.
    number of iteration done: number of iteration done to reach the results.
    cluster index: the cluster indices, which will be corresponding to the cluster controids.
    
    Solution Structure:
    To keep track of the distances a matrix X is prepared with k number of columns(required number of clusters).
    The number of rows in X is the number of items in the feature_sample(training data).
    
    Then X is filled with the distances from each of the initial cluster centroids.
    
    The initial centroids are taken considering items with minimum distances.
    
    The means of each of the attributes is the modified centroids. 
    
    """
    # necessary modules
    import random

    cluster_indices = None       # to store the cluster indices of the test data.


    fdata = feature_sample
    data = test_sample
    
    n, p = data.shape
    
    
    X = np.zeros([n, k])        # matrix X record the distances from centroids.
    X.fill(np.nan)

    # randomly choose k centroids
    random_centroids_row = random.sample(range(n), k)
    initial_centroids = fdata[random_centroids_row,]

    
    
    convergence = True
    
    iteration = 0


    make_vector = lambda X : np.squeeze(np.asarray(X)) # this function makes an two dimension n*1 matrix to n item vector.

    while(iteration < iter_conv and convergence):
        
        # Measure distance from each attribute of the feature and centroid and keep it to the matrix X                            

        for i in range(n):
            for j in range(k):
                X[i,j] = euclidean_distance(make_vector(fdata[i]), make_vector(initial_centroids[j,:])) 





        initial_cluster = [np.argmin(X[i]) for i in range(n)] # takes the index of the item which is closer to the centroids(the 
                                                              # column is the mark of initial centroids).
        
        
        # matrix modified_centroid to update the centroid.
        modified_centroid = np.zeros([k, p])
        modified_centroid.fill(np.nan)
        
        # take the mean of the initial clusters and update as modified centroids.

        for j in range(k):
            initial_cluster_features = [initial_cluster[i] == j for i in range(n)]
            modified_centroid[j, ] = np.mean(fdata[initial_cluster_features,], axis = 0)
            
        
        # check the convergenc.
        if (initial_centroids == modified_centroid).all():
            convergence = False
        else:
            initial_centroids = modified_centroid
            iteration += 1

    cluster_indices = initial_cluster

    return modified_centroid, iteration, cluster_indices


import math   

## Function for euclidean Distance
def euclidean_distance(x, y):
    # import math
    if len(x) != len(y):
        print('Length of two features/vectors are not same.')
    else:
        n = len(x)
        d2 = 0
        for i in range(n):
            d2 = d2 + (x[i] - y[i])**2
        d = math.sqrt(d2)
    
    return d

# Implementation with Iris Dataset

## Step 1: Load the Dataset

In [20]:
os.chdir(r"C:\Users\tahsi\OneDrive - University of Eastern Finland\Python Algorithm and Data Structure\GitHub\ml-algorithms-python-numpy\1. Codes")
Iris, d_map = load_text_data(file = "Iris.txt", delimiter = ",", header = False , label_col_is_str = True , label_col = 4)

In [21]:
# first 5 rows of the data. The 5th column was the label. The label was string. The function mapped it to integer. The map of label
# is stored in the d_map
print(Iris[0:5 ,:])

[[5.1 3.5 1.4 0.2 0. ]
 [4.9 3.  1.4 0.2 0. ]
 [4.7 3.2 1.3 0.2 0. ]
 [4.6 3.1 1.5 0.2 0. ]
 [5.  3.6 1.4 0.2 0. ]]


In [22]:
print(d_map)

{'Iris-setosa': 0, 'Iris-versicolor': 1, 'Iris-virginica': 2}


## Step 2: Split the Data to Train and Test

In [23]:
train_test_data = split_train_test(data = Iris, percentage = 50, label = 4)

train_data_feature = train_test_data.train_feature
train_data_label = train_test_data.train_label
test_feature = train_test_data.test_feature
true_label = train_test_data.test_label

## Step 3: Implementation of K means algorithm

In [24]:
centroids, iteration, cluster_indices = k_means(feature_sample = train_data_feature, test_sample = test_feature, k = 3, iter_conv = 100)

In [25]:
print(centroids)

[[5.93714286 2.76285714 4.45428571 1.45142857]
 [6.84285714 3.12142857 5.68571429 1.97857143]
 [4.98461538 3.43461538 1.43076923 0.21923077]]


In [26]:
print(iteration)

21


In [27]:
print(cluster_indices)

[1, 2, 1, 0, 0, 2, 2, 1, 2, 0, 0, 2, 1, 0, 2, 1, 2, 2, 1, 2, 2, 2, 0, 0, 0, 0, 1, 0, 0, 2, 0, 2, 1, 2, 2, 0, 0, 2, 0, 1, 1, 0, 0, 0, 0, 0, 2, 0, 1, 2, 2, 0, 0, 2, 1, 2, 2, 1, 0, 2, 0, 0, 1, 0, 0, 0, 2, 0, 0, 0, 2, 2, 0, 0, 0]
